# Cuaderno de jupyter para extraer parte del dataset original, ya que es muy grande y contiene algnos datos que no necesito

## Importación de librerias

In [1]:
import pandas as pd # tratamiento de archivos csv mediante dataframes
import os # acceso a archivos del sistema
from mutagen.mp3 import MP3 # se ven los metadatos de los archivos de audio para saber su duracion (mucho mas apido que librosa.load)
import shutil # utilizada para copiar y mover archivos

In [2]:
# Ruta al dataset descargado de mozilla
rutaDataset = '..\\..\\..\\Datasets\\cv-corpus-12.0-2022-12-07-es\\cv-corpus-12.0-2022-12-07\\es'
# Ruta a la carpeta del dataset que contiene los audios
rutaAudios = os.path.join(rutaDataset, "clips")
# Ruta en la cual se van a almacenar los audios extraidos para preprocesar y realizar el entrenamiento de la red
rutaDestinoEntrenamiento = "../audios/audiosEntrenamiento"
# Ruta en la cual se van a almacenar los audios extraidos para preprocesar y realizar los test de la red
rutaDestinoTest = "../audios/audiosTest"

# Se ven los archivos .tsv del dataset, para obtener sus datos
archivosTsv = []
for component in os.listdir(rutaDataset):
    if os.path.isfile(os.path.join(rutaDataset, component)):
        archivosTsv.append((component, os.path.join(rutaDataset, component)))

for archivo in archivosTsv:
    print(archivo)

('dev.tsv', '..\\..\\..\\Datasets\\cv-corpus-12.0-2022-12-07-es\\cv-corpus-12.0-2022-12-07\\es\\dev.tsv')
('invalidated.tsv', '..\\..\\..\\Datasets\\cv-corpus-12.0-2022-12-07-es\\cv-corpus-12.0-2022-12-07\\es\\invalidated.tsv')
('other.tsv', '..\\..\\..\\Datasets\\cv-corpus-12.0-2022-12-07-es\\cv-corpus-12.0-2022-12-07\\es\\other.tsv')
('reported.tsv', '..\\..\\..\\Datasets\\cv-corpus-12.0-2022-12-07-es\\cv-corpus-12.0-2022-12-07\\es\\reported.tsv')
('test.tsv', '..\\..\\..\\Datasets\\cv-corpus-12.0-2022-12-07-es\\cv-corpus-12.0-2022-12-07\\es\\test.tsv')
('train.tsv', '..\\..\\..\\Datasets\\cv-corpus-12.0-2022-12-07-es\\cv-corpus-12.0-2022-12-07\\es\\train.tsv')
('trainMas5.csv', '..\\..\\..\\Datasets\\cv-corpus-12.0-2022-12-07-es\\cv-corpus-12.0-2022-12-07\\es\\trainMas5.csv')
('validated.tsv', '..\\..\\..\\Datasets\\cv-corpus-12.0-2022-12-07-es\\cv-corpus-12.0-2022-12-07\\es\\validated.tsv')


In [3]:
def cargarCSV(ruta):
    # De las columnas del archivo solo se cargan el id y el genero del cliente y la ruta al audio, el resto no lo necesito
    archivo = pd.read_csv(ruta[1], low_memory=False, sep="\t", header=0).loc[:, ('client_id', 'path', 'gender')]
    # Se eliminan los valores NaN
    archivo = archivo.dropna()
    # Se eliminan los valores que no sean 'male' o 'female' en el genero
    archivo = archivo[archivo['gender'].isin(['male', 'female'])]
    print(f'Cantidad de filas utiles del archivo {ruta[0]}: {len(archivo)}')

    return archivo

In [4]:
def eliminarAudiosMenos5Seg(rutaAudios, rutaDestino, dataframe, nombre, inicio, cantidadRegistros):
    fila = 0
    if cantidadRegistros == 0:
        cantidadRegistros = len(dataframe)

    print(f'Recorriendo dataframe: {nombre}')

    for audio in dataframe.iloc[inicio:cantidadRegistros]['path']: # Se recorre cada archivo de audio
        
        if os.path.join(rutaAudios, audio).endswith('.mp3'):
            duracion_segundos = MP3(os.path.join(rutaAudios, audio)).info.length

            # Si el archivo dura 5 o mas segundos, se considera valido para entrenar la red y se copia a la carpeta destino
            if duracion_segundos >= 5:
                shutil.copy(os.path.join(rutaAudios, audio), os.path.join(rutaDestino, audio))
            else: # Si no, se elimina del dataframe, ya que este se utilizara mas adelante
                dataframe = dataframe.drop(dataframe[dataframe['path']==audio].index)
        
        fila+= 1
        print(fila, end="\r")

    # Se reinician los indices, para que vayan ordenados sin hueco entre ellos
    dataframe.reset_index(inplace = True, drop = True)
    
    return dataframe

In [5]:
def crearCsv(dataframe, audiosCarpetaDestino, nombreCSV, inicio, cantidadRegistros):
    datos = {'client_id': [], 'path': [], 'gender': []}

    if cantidadRegistros == 0:
        cantidadRegistros = len(dataframe)

    for i in range(inicio, cantidadRegistros):
        if(dataframe.iloc[i]['path'] in audiosCarpetaDestino):
            datos['client_id'].append(dataframe.iloc[i]['client_id'])
            datos['path'].append(dataframe.iloc[i]['path'])
            datos['gender'].append(dataframe.iloc[i]['gender'])

    pd.DataFrame(datos).to_csv(nombreCSV, header=True, index=False)

In [6]:
testTsv = cargarCSV(archivosTsv[4])
trainTsv = cargarCSV(archivosTsv[5])
validatedTsv = cargarCSV(archivosTsv[7])

Cantidad de filas utiles del archivo test.tsv: 3029
Cantidad de filas utiles del archivo train.tsv: 234509
Cantidad de filas utiles del archivo validated.tsv: 267103


In [7]:
trainTsv = eliminarAudiosMenos5Seg(rutaAudios, rutaDestinoEntrenamiento, trainTsv, archivosTsv[5][0], 0, 0)
validatedTsv = eliminarAudiosMenos5Seg(rutaAudios, rutaDestinoEntrenamiento, validatedTsv, archivosTsv[7][0], 0, 0)
testTsv = eliminarAudiosMenos5Seg(rutaAudios, rutaDestinoTest, testTsv, archivosTsv[4][0], 0, 0)

Recorriendo dataframe: train.tsv
Recorriendo dataframe: validated.tsv
Recorriendo dataframe: test.tsv


In [8]:
audiosValidosEntrenamiento = os.listdir(rutaDestinoEntrenamiento)
audiosValidosTest = os.listdir(rutaDestinoTest)

In [ ]:
crearCsv(trainTsv, audiosValidosEntrenamiento, "../csv/archivoTrain.csv", 0, 0)
crearCsv(validatedTsv, audiosValidosEntrenamiento, "../csv/archivoValidated.csv", 0, 0)
crearCsv(testTsv, audiosValidosTest, "../csv/archivoTest.csv", 0, 0)